In [1]:
import geemap
import ee

/home/zyang91/anaconda3/envs/geosptial/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="ee-zhanchaoyang")

In [4]:
m=geemap.Map(center=(40,-100),zoom=4)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [5]:
geometry = ee.Geometry.Polygon([
  [74.322, 14.981],
  [74.322, 14.765],
  [74.648, 14.765],
  [74.648, 14.980]
])

In [6]:
m.add_basemap('SATELLITE')

In [7]:
m.addLayer(geometry, {}, "geometry")
m.centerObject(geometry)

In [8]:
startDate = ee.Date.fromYMD(2022, 1, 1)

In [9]:
endDate = ee.Date.fromYMD(2023, 1, 1)

In [10]:
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

In [11]:
embeddingsFiltered = embeddings.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

## Visulization

In [12]:
embeddings_vis=embeddingsFiltered.median().clip(geometry)

In [13]:
vis_params = {
    "bands": ["A00", "A01", "A02"],
    "min": -0.5,
    "max":0.5,
}

In [14]:
m.addLayer(embeddings_vis, vis_params, "embeddings")
m

Map(center=[14.873000000000001, 74.485], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

## Tasks continue

In [15]:
# check the projections 
embeddingsProjection = ee.Image(embeddingsFiltered.first()).select(0).projection()

In [16]:
embeddingsProjection

In [17]:
embeddingsImage = embeddingsFiltered.mosaic().setDefaultProjection(embeddingsProjection)

In [18]:
gedi = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')

In [19]:
def qualityMask(image):
    mask= image.updateMask(image.select('l4_quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))
    return mask

In [20]:
def errorMask(image):
    relative_se = image.select('agbd_se').divide(image.select('agbd'))
    error_mask= image.updateMask(relative_se.lte(0.5))
    return error_mask

In [21]:
def SlopeMask(image):
    glo30 = ee.ImageCollection('COPERNICUS/DEM/GLO30')
    glo30Filtered = glo30.filter(ee.Filter.bounds(geometry)).select('DEM')
    demProj = glo30Filtered.first().select(0).projection()
    elevation = glo30Filtered.mosaic().rename('dem').setDefaultProjection(demProj)
    slope = ee.Terrain.slope(elevation)
    slope_mask = image.updateMask(slope.lte(30))
    return slope_mask

In [22]:
gediFiltered = gedi.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

In [23]:
gediProjection = ee.Image(gediFiltered.first()).select('agbd').projection()

In [24]:
gedi_processed = (gediFiltered.map(qualityMask).map(errorMask).map(SlopeMask))

In [25]:
gediMosaic = gedi_processed.mosaic().select('agbd').setDefaultProjection(gediProjection)

In [26]:
gediVis = {
  min: 0,
  max: 200,
  'palette': ['#edf8fb', '#b2e2e2', '#66c2a4', '#2ca25f', '#006d2c'],
  'bands': ['agbd'],
}

In [27]:
m.addLayer(gedi_processed, gediVis, "gedi")
m

Map(center=[14.873000000000001, 74.485], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [28]:
gridScale = 100

In [29]:
gridProjection = ee.Projection('EPSG:3857').atScale(gridScale)

In [30]:
stacked = embeddingsImage.addBands(gediMosaic)

In [31]:
stacked = stacked.resample('bilinear')

In [32]:
stackedResampled = (
    stacked
    .reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=1024
    )
    .reproject(
        crs=gridProjection
    )
)

In [33]:
stackedResampled = stackedResampled.updateMask(stackedResampled.mask().gt(0))

In [34]:
export_image = stackedResampled.clip(geometry)

In [35]:
geemap.ee_export_image_to_drive(
    export_image,
    description='GEDI_Mosaic_Export',
    folder='EarthEngine',         # Change to your Google Drive folder
    fileNamePrefix='gedi_mosaic', # No ".tif" extension
    scale=gridScale,
    region=geometry,
    maxPixels=1e10
)

In [36]:
predictors = embeddingsImage.bandNames()

In [37]:
predicted = gediMosaic.bandNames().get(0)

In [38]:
print('predictors', predictors.getInfo())

predictors ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63']


In [39]:
print('predicted', predicted.getInfo())

predicted agbd


In [40]:
predictorImage = stackedResampled.select(predictors)

In [41]:
predictedImage = stackedResampled.select([predicted])

In [42]:
classMask = predictedImage.mask().toInt().rename('class')

In [43]:
numSamples = 1000

In [44]:
training = stackedResampled.addBands(classMask).stratifiedSample(
    numPoints=numSamples,
    classBand='class',
    region=geometry,
    scale=gridScale,
    classValues=[0, 1],
    classPoints=[0, numSamples],
    dropNulls=True,
    tileScale=16
)


In [45]:
model = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION').train(
    features= training,
    classProperty= predicted,
    inputProperties= predictors
  )


In [46]:
predicted = training.classify(
  classifier= model,
  outputName= 'agbd_predicted'
)

In [47]:
def calculatermse(inputs):
    observed=ee.Array(inputs.aggregate_array('agbd'))
    predicted=ee.Array(inputs.aggregate_array('agbd_predicted'))
    rmse=observed.subtract(predicted).pow(2).reduce(ee.Reducer.mean(), [0]).sqrt().get([0])
    return rmse

In [48]:
rmse = calculatermse(predicted)
print('RMSE', rmse.getInfo())

RMSE 28.506312098165626


## Plot

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt